# CoxPH tests

In [11]:
# nuclio: ignore
import nuclio

In [12]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [13]:
import os
import pandas as pd
from mlrun.datastore import DataItem
from mlrun.artifacts import get_model
from cloudpickle import load
from mlrun.mlutils.models import eval_class_model

def cox_test(
    context,
    models_path: DataItem, 
    test_set: DataItem,
    label_column: str,
    plots_dest: str = "plots",
    model_evaluator = None
) -> None:
    """Test one or more classifier models against held-out dataset
    
    Using held-out test features, evaluates the peformance of the estimated model
    
    Can be part of a kubeflow pipeline as a test step that is run post EDA and 
    training/validation cycles
    
    :param context:         the function context
    :param model_file:      model artifact to be tested
    :param test_set:        test features and labels
    :param label_column:    column name for ground truth labels
    :param score_method:    for multiclass classification
    :param plots_dest:      dir for test plots
    :param model_evaluator: WIP: specific method to generate eval, passed in as string
                            or available in this folder
    """  
    xtest = test_set.as_df()
    ytest = xtest.pop(label_column)
    
    model_file, model_obj, _ = get_model(models_path.url, suffix='.pkl')
    model_obj = load(open(str(model_file), "rb"))

    try:
        # there could be different eval_models, type of model (xgboost, tfv1, tfv2...)
        if not model_evaluator:
            # binary and multiclass
            eval_metrics = eval_class_model(context, xtest, ytest, model_obj)

        # just do this inside log_model?
        model_plots = eval_metrics.pop("plots")
        model_tables = eval_metrics.pop("tables")
        for plot in model_plots:
            context.log_artifact(plot, local_path=f"{plots_dest}/{plot.key}.html")
        for tbl in model_tables:
            context.log_artifact(tbl, local_path=f"{plots_dest}/{plot.key}.csv")

        context.log_results(eval_metrics)
    except:
        #dummy log:
        context.log_dataset("cox-test-summary", df=model_obj.summary, index=True, format="csv")
        context.logger.info("cox tester not implemented")

In [14]:
# nuclio: end-code

In [16]:
task_params = {
    "name" : "tasks cox test",
    "params": {
        "label_column"  : "labels",
        "plots_dest"    : "churn/test/plots"}}

In [17]:
DATA_URL = "https://raw.githubusercontent.com/yjb-ds/testdata/master/demos/churn/churn-tests.csv"

## Run Locally

In [19]:
from mlrun import run_local, NewTask, mlconf

run = run_local(NewTask(**task_params),
                handler=cox_test,
                inputs={"test_set": DATA_URL,
                        "models_path"   : "models/cox"},
               workdir=mlconf.artifact_path+"/churn")


[mlrun] 2020-06-14 13:09:54,707 starting run tasks cox test uid=6284f5f5e6d14a969ba81addcfaf200f  -> http://mlrun-api:8080
[mlrun] 2020-06-14 13:09:55,249 log artifact cox-test-summary at /User/artifacts/cox-test-summary.csv, size: 3395, db: Y
[mlrun] 2020-06-14 13:09:55,250 cox tester not implemented



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...cfaf200f,0,Jun 14 13:09:54,completed,tasks cox test,v3io_user=adminkind=handlerowner=adminhost=jupyter-7b44c8d958-kklf7,test_setmodels_path,label_column=labelsplots_dest=churn/test/plots,,cox-test-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6284f5f5e6d14a969ba81addcfaf200f --project default , !mlrun logs 6284f5f5e6d14a969ba81addcfaf200f --project default
[mlrun] 2020-06-14 13:09:55,307 run executed, status=completed


## Run Remotely

In [ ]:
from mlrun import import_function
from mlrun.platforms.other import auto_mount

GPU = False

fn = import_function("hub://coxph_test")
if GPU:
    fn.image = "mlrun/ml-models-gpu"
fn.apply(auto_mount())

run = fn.run(
    NewTask(**task_params),
    inputs={
        "test_set"    : DATA_URL,
        "models_path" : "models/cox"},
    workdir=os.path.join(mlconf.artifact_path, "churn"))